# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
C:\Users\Quan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Quan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Quan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Quan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [7]:
np.random.seed(123)
baseline_model = Sequential()
# Hidden layers and Output layer
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
baseline_model.add(layers.Dense(50, activation='relu'))
baseline_model.add(layers.Dense(1, activation='linear'))
# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])
# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))


Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 147us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
1051/1051 [==============================] - 0s 49us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
1051/1051 [==============================] - 0s 48us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
1051/1051 [==============================] - 0s 48us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
1051/1051 [==============================] - 0s 50us/step - loss: nan -

1051/1051 [==============================] - 0s 49us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 65/150
1051/1051 [==============================] - 0s 48us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 66/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 67/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 68/150
1051/1051 [==============================] - 0s 46us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
1051/1051 [==============================] - 0s 46us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
1051/1051 [==============================] - 0s 48us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150


1051/1051 [==============================] - 0s 48us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 130/150
1051/1051 [==============================] - 0s 49us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 131/150
1051/1051 [==============================] - 0s 49us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 132/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 133/150
1051/1051 [==============================] - 0s 44us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 134/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 135/150
1051/1051 [==============================] - 0s 46us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
1051/1051 [==============================] - 0s 45us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [8]:
# Assign the column names of all numeric columns to:
numeric_columns = X_train_numeric.columns 
# Instantiate StandardScaler
ss_X = StandardScaler()
# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)
# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)
# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [9]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))
# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [10]:
# Train the model 
normalized_input_model.fit(X_train, y_train, batch_size=32, 
                           epochs=150, validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 133us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
1051/1051 [==============================] - 0s 49us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
1051/1051 [==============================] - 0s 48us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
1051/1051 [==============================] - 0s 49us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
1051/1051 [==============================] - 0s 49us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
1051/1051 [==============================] - 0s 49us/step - loss: nan - 

1051/1051 [==============================] - 0s 49us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 67/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 68/150
1051/1051 [==============================] - 0s 55us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
1051/1051 [==============================] - 0s 48us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
1051/1051 [==============================] - 0s 46us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150


1051/1051 [==============================] - 0s 47us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 132/150
1051/1051 [==============================] - 0s 49us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 133/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 134/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 135/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
1051/1051 [==============================] - 0s 45us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
1051/1051 [==============================] - 0s 49us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
1051/1051 [==============================] - 0s 48us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [11]:
# Instantiate StandardScaler
ss_y = StandardScaler()
# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)
# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [12]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 
# Train the model
normalized_model.fit(X_train, y_train_scaled,
                     batch_size=32, epochs=150,
                     validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 140us/step - loss: 0.4368 - mse: 0.4368 - val_loss: 0.1992 - val_mse: 0.1992
Epoch 2/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.2274 - mse: 0.2274 - val_loss: 0.1613 - val_mse: 0.1613
Epoch 3/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.1842 - mse: 0.1842 - val_loss: 0.1588 - val_mse: 0.1588
Epoch 4/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.1642 - mse: 0.1642 - val_loss: 0.1376 - val_mse: 0.1376
Epoch 5/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.1480 - mse: 0.1480 - val_loss: 0.1321 - val_mse: 0.1321
Epoch 6/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.1330 - mse: 0.1330 - val_loss: 0.1277 - val_mse: 0.1277
Epoch 7/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.1266 - mse: 0.1266 - val_loss: 0.1307 - val_mse: 0.1307


1051/1051 [==============================] - 0s 50us/step - loss: 0.0259 - mse: 0.0259 - val_loss: 0.1068 - val_mse: 0.1068
Epoch 61/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0254 - mse: 0.0254 - val_loss: 0.1048 - val_mse: 0.1048
Epoch 62/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0249 - mse: 0.0249 - val_loss: 0.1101 - val_mse: 0.1101
Epoch 63/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0248 - mse: 0.0248 - val_loss: 0.1068 - val_mse: 0.1068
Epoch 64/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0240 - mse: 0.0240 - val_loss: 0.1078 - val_mse: 0.1078
Epoch 65/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1064 - val_mse: 0.1064
Epoch 66/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1078 - val_mse: 0.1078
Epoch 67/150
1051/1051 [==============================

Epoch 120/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1161 - val_mse: 0.1161
Epoch 121/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1163 - val_mse: 0.1163
Epoch 122/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1163 - val_mse: 0.1163
Epoch 123/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1166 - val_mse: 0.1166
Epoch 124/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1145 - val_mse: 0.1145
Epoch 125/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1170 - val_mse: 0.1170
Epoch 126/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1168 - val_mse: 0.1168
Epoch 127/150
1051/1051 [=========

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [13]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 20us/step


[0.0077378492191837985, 0.007737850304692984]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [14]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 30us/step


[0.11930205092230677, 0.11930206418037415]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [15]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)
# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)
# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

27142.851445299155

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [16]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='he_normal'))
# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))
# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))
# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])
# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 152us/step - loss: 0.5083 - mse: 0.5083 - val_loss: 0.1699 - val_mse: 0.1699
Epoch 2/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.2217 - mse: 0.2217 - val_loss: 0.1432 - val_mse: 0.1432
Epoch 3/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.1798 - mse: 0.1798 - val_loss: 0.1378 - val_mse: 0.1378
Epoch 4/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.1569 - mse: 0.1569 - val_loss: 0.1250 - val_mse: 0.1250
Epoch 5/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.1410 - mse: 0.1410 - val_loss: 0.1195 - val_mse: 0.1195
Epoch 6/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.1265 - mse: 0.1265 - val_loss: 0.1149 - val_mse: 0.1149
Epoch 7/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.1191 - mse: 0.1191 - val_loss: 0.1168 - val_mse: 0.1168


1051/1051 [==============================] - 0s 49us/step - loss: 0.0250 - mse: 0.0250 - val_loss: 0.1112 - val_mse: 0.1112
Epoch 61/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0246 - mse: 0.0246 - val_loss: 0.1123 - val_mse: 0.1123
Epoch 62/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0245 - mse: 0.0245 - val_loss: 0.1152 - val_mse: 0.1152
Epoch 63/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0243 - mse: 0.0243 - val_loss: 0.1121 - val_mse: 0.1121
Epoch 64/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0241 - mse: 0.0241 - val_loss: 0.1131 - val_mse: 0.1131
Epoch 65/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1130 - val_mse: 0.1130
Epoch 66/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1150 - val_mse: 0.1150
Epoch 67/150
1051/1051 [==============================

Epoch 120/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1216 - val_mse: 0.1216
Epoch 121/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1211 - val_mse: 0.1211
Epoch 122/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1210 - val_mse: 0.1210
Epoch 123/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.1210 - val_mse: 0.1210
Epoch 124/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1205 - val_mse: 0.1205
Epoch 125/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1249 - val_mse: 0.1249
Epoch 126/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1217 - val_mse: 0.1217
Epoch 127/150
1051/1051 [=========

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 20us/step


[0.008180097817708265, 0.008180097676813602]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`) 

In [18]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 34us/step


[0.12427428012055136, 0.12427427619695663]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [19]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', 
                             input_shape=(n_features,), 
                             kernel_initializer='lecun_normal'))
# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))
# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))
# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])
# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 157us/step - loss: 0.4769 - mse: 0.4769 - val_loss: 0.1461 - val_mse: 0.1461
Epoch 2/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.2096 - mse: 0.2096 - val_loss: 0.1230 - val_mse: 0.1230
Epoch 3/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.1698 - mse: 0.1698 - val_loss: 0.1168 - val_mse: 0.1168
Epoch 4/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.1479 - mse: 0.1479 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 5/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.1331 - mse: 0.1331 - val_loss: 0.1059 - val_mse: 0.1059
Epoch 6/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.1197 - mse: 0.1197 - val_loss: 0.1051 - val_mse: 0.1051
Epoch 7/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.1124 - mse: 0.1124 - val_loss: 0.1091 - val_mse: 0.1091


1051/1051 [==============================] - 0s 50us/step - loss: 0.0219 - mse: 0.0219 - val_loss: 0.1196 - val_mse: 0.1196
Epoch 61/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1204 - val_mse: 0.1204
Epoch 62/150
1051/1051 [==============================] - 0s 54us/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.1248 - val_mse: 0.1248
Epoch 63/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0215 - mse: 0.0215 - val_loss: 0.1205 - val_mse: 0.1205
Epoch 64/150
1051/1051 [==============================] - 0s 69us/step - loss: 0.0213 - mse: 0.0213 - val_loss: 0.1235 - val_mse: 0.1235
Epoch 65/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.0207 - mse: 0.0207 - val_loss: 0.1215 - val_mse: 0.1215
Epoch 66/150
1051/1051 [==============================] - 0s 84us/step - loss: 0.0197 - mse: 0.0197 - val_loss: 0.1252 - val_mse: 0.1252
Epoch 67/150
1051/1051 [==============================

Epoch 120/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1313 - val_mse: 0.1313
Epoch 121/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1306 - val_mse: 0.1306
Epoch 122/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1310 - val_mse: 0.1310
Epoch 123/150
1051/1051 [==============================] - 0s 54us/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1308 - val_mse: 0.1308
Epoch 124/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1293 - val_mse: 0.1293
Epoch 125/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1359 - val_mse: 0.1359
Epoch 126/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1316 - val_mse: 0.1316
Epoch 127/150
1051/1051 [=========

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 21us/step


[0.007117834705575861, 0.0071178339421749115]

Evaluate the model (`lecun_model`) on validate data (`X_val` and `y_val_scaled`) 

In [21]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 38us/step


[0.1344752639546254, 0.13447527587413788]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [22]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))
# Compile the model
rmsprop_model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])
# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 197us/step - loss: 0.3205 - mse: 0.3205 - val_loss: 0.1370 - val_mse: 0.1370
Epoch 2/150
1051/1051 [==============================] - 0s 61us/step - loss: 0.1666 - mse: 0.1666 - val_loss: 0.1106 - val_mse: 0.1106
Epoch 3/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.1242 - mse: 0.1242 - val_loss: 0.1228 - val_mse: 0.1228
Epoch 4/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0992 - mse: 0.0992 - val_loss: 0.1091 - val_mse: 0.1091
Epoch 5/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0888 - mse: 0.0888 - val_loss: 0.1000 - val_mse: 0.1000
Epoch 6/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0661 - mse: 0.0661 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 7/150
1051/1051 [==============================] - 0s 60us/step - loss: 0.0562 - mse: 0.0562 - val_loss: 0.1144 - val_mse: 0.1144


1051/1051 [==============================] - 0s 57us/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.0931 - val_mse: 0.0931
Epoch 61/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.0992 - val_mse: 0.0992
Epoch 62/150
1051/1051 [==============================] - 0s 61us/step - loss: 0.0050 - mse: 0.0050 - val_loss: 0.0859 - val_mse: 0.0859
Epoch 63/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0061 - mse: 0.0061 - val_loss: 0.1023 - val_mse: 0.1023
Epoch 64/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.0081 - mse: 0.0081 - val_loss: 0.0899 - val_mse: 0.0899
Epoch 65/150
1051/1051 [==============================] - 0s 60us/step - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0894 - val_mse: 0.0894
Epoch 66/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0065 - mse: 0.0065 - val_loss: 0.0880 - val_mse: 0.0880
Epoch 67/150
1051/1051 [==============================

1051/1051 [==============================] - 0s 59us/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.0888 - val_mse: 0.0888
Epoch 120/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.0046 - mse: 0.0046 - val_loss: 0.0995 - val_mse: 0.0995
Epoch 121/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.0047 - mse: 0.0047 - val_loss: 0.0923 - val_mse: 0.0923
Epoch 122/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.1098 - val_mse: 0.1098
Epoch 123/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0046 - mse: 0.0046 - val_loss: 0.0919 - val_mse: 0.0919
Epoch 124/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.0036 - mse: 0.0036 - val_loss: 0.0898 - val_mse: 0.0898
Epoch 125/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0039 - mse: 0.0039 - val_loss: 0.0844 - val_mse: 0.0844
Epoch 126/150
1051/1051 [=======================

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 24us/step


[0.006984021760998285, 0.006984021048992872]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [24]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 38us/step


[0.08595666437349618, 0.08595666289329529]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [26]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))
# Compile the model
adam_model.compile(optimizer='Adam', loss='mse', metrics=['mse'])
# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 244us/step - loss: 0.3726 - mse: 0.3726 - val_loss: 0.1645 - val_mse: 0.1645
Epoch 2/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.1568 - mse: 0.1568 - val_loss: 0.1315 - val_mse: 0.1315
Epoch 3/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.1105 - mse: 0.1105 - val_loss: 0.1026 - val_mse: 0.1026
Epoch 4/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0852 - mse: 0.0852 - val_loss: 0.1022 - val_mse: 0.1022
Epoch 5/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0729 - mse: 0.0729 - val_loss: 0.1027 - val_mse: 0.1027
Epoch 6/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0560 - mse: 0.0560 - val_loss: 0.0965 - val_mse: 0.0965
Epoch 7/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.0454 - mse: 0.0454 - val_loss: 0.1140 - val_mse: 0.1140


1051/1051 [==============================] - 0s 67us/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.1141 - val_mse: 0.1141
Epoch 61/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.0982 - val_mse: 0.0982
Epoch 62/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.1093 - val_mse: 0.1093
Epoch 63/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.1017 - val_mse: 0.1017
Epoch 64/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0028 - mse: 0.0028 - val_loss: 0.0974 - val_mse: 0.0974
Epoch 65/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0030 - mse: 0.0030 - val_loss: 0.1075 - val_mse: 0.1075
Epoch 66/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0028 - mse: 0.0028 - val_loss: 0.0987 - val_mse: 0.0987
Epoch 67/150
1051/1051 [==============================

Epoch 119/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0028 - mse: 0.0028 - val_loss: 0.0912 - val_mse: 0.0912
Epoch 120/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0022 - mse: 0.0022 - val_loss: 0.0868 - val_mse: 0.0868
Epoch 121/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0020 - mse: 0.0020 - val_loss: 0.0890 - val_mse: 0.0890
Epoch 122/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0892 - val_mse: 0.0892
Epoch 123/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0887 - val_mse: 0.0887
Epoch 124/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.0882 - val_mse: 0.0882
Epoch 125/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0015 - mse: 0.0015 - val_loss: 0.0932 - val_mse: 0.0932
Epoch 126/150
1051/1051 [=========

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 24us/step


[0.001307930714706868, 0.0013079307973384857]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [28]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 38us/step


[0.08409003244833348, 0.08409002423286438]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [29]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 41us/step


[0.1426223920632715, 0.14262238144874573]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [30]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)
# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)
# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

29677.364672382788

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.